In [ ]:
import pandas as pd
from datetime import timedelta
from scipy.stats import skew
from ast import literal_eval
import numpy as np
import ast

In [ ]:
icustays = pd.read_csv("../data/mimic-iv-3.1/icu/icustays.csv")
admissions = pd.read_csv("../data/mimic-iv-3.1/hosp/admissions.csv")
patients = pd.read_csv("../data/mimic-iv-3.1/hosp/patients.csv")

In [ ]:
# filter patients under 18 years old

icustays = icustays.merge(patients[['subject_id', 'anchor_age']], on='subject_id', how='left')
icustays = icustays[icustays['anchor_age'] >= 18]
print(f"Number of adult ICU stays: {len(icustays)}")

In [ ]:
# filter out stays under 24 hours

icustays = icustays[(icustays['los'] >= 1)]
print(f"Number of ICU stays with LOS over 1 day: {len(icustays)}")

In [ ]:
# filter stays that resulted in death

icustays = icustays.merge(admissions[['hadm_id', 'hospital_expire_flag']], on='hadm_id', how='left')
icustays = icustays[icustays['hospital_expire_flag'] == 0]
print(f"Number of ICU stays excluding deaths: {len(icustays)}")

In [ ]:
# filter out multiple icu stays for single hospital admission

icustays = icustays.drop_duplicates(subset='hadm_id', keep=False)
icustays.to_csv('icustays_filtered.csv', index=False)
print('Number of ICU stays excluding multiple admissions:', len(icustays))

In [ ]:
# filter labevents

hadm_id_set = set(icustays['hadm_id'])

chunksize = 50000

filtered_chunks = []

for chunk in pd.read_csv('../data/mimic-iv-3.1/hosp/labevents.csv', chunksize=chunksize):
    filtered_chunk = chunk[chunk['hadm_id'].isin(hadm_id_set)]
    filtered_chunks.append(filtered_chunk)

filtered_labevents = pd.concat(filtered_chunks)
filtered_labevents.to_csv('labevents_filtered.csv', index=False)


In [ ]:
# filter chartevents

hadm_id_set = set(icustays['hadm_id'])

chunksize = 50000

output_file = 'chartevents_filtered.csv'
first_chunk = True  

for chunk in pd.read_csv('../data/mimic-iv-3.1/icu/chartevents.csv', chunksize=chunksize):
    filtered_chunk = chunk[chunk['hadm_id'].isin(hadm_id_set)]

    filtered_chunk.to_csv(output_file, mode='a', index=False, header=first_chunk)
    first_chunk = False  

In [ ]:
# extract relevant chartevents based on itemid

chartevents_itemids = {
    "Heart Rate (HR)": 220045,
    "O2 Saturation (SpO2)": 220277,
    "Respiratory Rate (RR)": 220210,
    "Body Temperature (Fahrenheit)": 223761,
    "GCS - Eye Opening": 220739,
    "GCS - Verbal Response": 223900,
    "GCS - Motor Response": 223901
}

selected_itemids = [220045, 220277, 220210, 223761, 220739, 223900, 223901]

chartevents_file = 'chartevents_filtered.csv'

chunksize = 50000

output_file = 'chartevents_filtered2.csv'

write_header = True

for chunk in pd.read_csv(chartevents_file, chunksize=chunksize, on_bad_lines='skip'):
    filtered_chunk = chunk[chunk['itemid'].isin(selected_itemids)]
    
    filtered_chunk.to_csv(output_file, mode='a', index=False, header=write_header)
    write_header = False

print(f"Filtered data has been saved to {output_file}.")


In [ ]:
# extract relevant labevents

labevents_itemids = {
    "Anion Gap": 50868,
    "Bicarbonate": 50882,
    "Chloride": 50902,
    "Creatinine": 50912,
    "Glucose": 50931,
    "Sodium": 50983,
    "Magnesium": 50960,
    "Potassium": 50971,
    "Phosphate": 50970,
    "Urea Nitrogen": 51006,
    "Hematocrit": 51221,
    "Hemoglobin": 51222,
    "MCH": 51248,
    "MCHC": 51249,
    "MCV": 51250,
    "Red Blood Cells": 51279,
    "White Blood Cells": 51301,
    "RDW": 51277,
    "Platelets": 51265
}

selected_itemids = [50868, 50882, 50902, 50912, 50931, 50983, 50960, 50971, 50970, 
                    51006, 51221, 51222, 51248, 51249, 51250, 51279, 51301, 51277, 51265]

chartevents_file = 'labevents_filtered.csv'

chunksize = 50000

output_file = 'labevents_filtered2.csv'

write_header = True

for chunk in pd.read_csv(chartevents_file, chunksize=chunksize):
    filtered_chunk = chunk[chunk['itemid'].isin(selected_itemids)]
    
    filtered_chunk.to_csv(output_file, mode='a', index=False, header=write_header)
    write_header = False

print(f"Filtered data has been saved to {output_file}.")

In [ ]:
# merge with intime and outtime

icustays = pd.read_csv('icustays_filtered.csv')
icu_times = icustays[['hadm_id', 'intime', 'outtime']]

def process_large_merge(input_file, output_file, icu_times):
    chunksize = 500000 
    write_header = True
    icu_times['hadm_id'] = icu_times['hadm_id'].astype(int)

    for chunk in pd.read_csv(input_file, chunksize=chunksize, low_memory=False):
        chunk = chunk.merge(icu_times, on='hadm_id', how='inner')  
        chunk['hadm_id'] = chunk['hadm_id'].astype(int)
        chunk.to_csv(output_file, mode='a', index=False, header=write_header) 
        write_header = False 
process_large_merge('chartevents_filtered2.csv', 'chartevents_times.csv', icu_times)

process_large_merge('labevents_filtered2.csv', 'labevents_times.csv', icu_times)

In [ ]:
chartevents = pd.read_csv('chartevents_times.csv', usecols=['hadm_id'])
print('Chartevents, unique admissions:', chartevents['hadm_id'].nunique())

labevents = pd.read_csv('labevents_times.csv', usecols=['hadm_id'])
print('Labevents, unique admissions:', labevents['hadm_id'].nunique())

In [ ]:
# keep only measurements within the first 24 hours

def filter_dataframe_in_chunks(input_file, output_file):
    chunksize = 50000
    write_header = True 

    for chunk in pd.read_csv(input_file, chunksize=chunksize):
        chunk['charttime'] = pd.to_datetime(chunk['charttime'])
        chunk['intime'] = pd.to_datetime(chunk['intime'])

        chunk['time_diff'] = (chunk['charttime'] - chunk['intime']).dt.total_seconds() / 3600

        filtered_chunk = chunk[(chunk['time_diff'] >= 0) & (chunk['time_diff'] <= 24)]

        filtered_chunk.to_csv(output_file, mode='a', index=False, header=write_header)
        write_header = False  

    print(f"Filtered data saved to {output_file}.")

filter_dataframe_in_chunks('chartevents_times.csv', 'chartevents24.csv')

filter_dataframe_in_chunks('labevents_times.csv', 'labevents24.csv')


In [ ]:
chartevents = pd.read_csv('chartevents24.csv', usecols=['hadm_id'])
print('Chartevents, unique admissions:', chartevents['hadm_id'].nunique())
print('Chartevents, length:', len(chartevents.index))

labevents = pd.read_csv('labevents24.csv', usecols=['hadm_id'])
print('Labevents, unique admissions:', labevents['hadm_id'].nunique())
print('Labevents, length:', len(labevents.index))

In [ ]:
# create lists for chartevents and labevents

def group_measurements(df):
    df['value'] = df['valuenum'].fillna(df['value'])
    grouped_df = df.groupby(['subject_id', 'hadm_id', 'itemid'])['value'].agg(list).reset_index()

    return grouped_df

chartevents = pd.read_csv('chartevents24.csv') 
chartevents_list = group_measurements(chartevents)
chartevents_list.to_csv('chartevents_list.csv', index=False)

labevents = pd.read_csv('labevents24.csv') 
labevents_list = group_measurements(labevents)
labevents_list.to_csv('labevents_list.csv', index=False)

In [ ]:
# check for the % of missing itemids

def calculate_missing_itemids_per_variable(labevents_file, chartevents_file, labevents_itemids, chartevents_itemids):

  labevents_df = pd.read_csv(labevents_file)
  chartevents_df = pd.read_csv(chartevents_file)

  all_itemids = {**labevents_itemids, **chartevents_itemids} 

  missing_percentages = {}

  for item_name, itemid in all_itemids.items():
    if item_name in labevents_itemids:
      filtered_df = labevents_df[labevents_df['itemid'] == labevents_itemids[item_name]]
      total_admissions = labevents_df['hadm_id'].nunique()
      admissions_with_item = filtered_df['hadm_id'].nunique()
      missing_percentages[item_name] = ((total_admissions - admissions_with_item) / total_admissions) * 100
    elif item_name in chartevents_itemids:
      filtered_df = chartevents_df[chartevents_df['itemid'] == chartevents_itemids[item_name]]
      total_admissions = chartevents_df['hadm_id'].nunique()
      admissions_with_item = filtered_df['hadm_id'].nunique()
      missing_percentages[item_name] = ((total_admissions - admissions_with_item) / total_admissions) * 100

  return missing_percentages

labevents_file = 'labevents_list.csv'
chartevents_file = 'chartevents_list.csv'

chartevents_itemids = {
    "Heart Rate (HR)": 220045,
    "O2 Saturation (SpO2)": 220277,
    "Respiratory Rate (RR)": 220210,
    "Body Temperature (Fahrenheit)": 223761,
    "GCS - Eye Opening": 220739,
    "GCS - Verbal Response": 223900,
    "GCS - Motor Response": 223901
}

labevents_itemids = {
    "Anion Gap": 50868,
    "Bicarbonate": 50882,
    "Chloride": 50902,
    "Creatinine": 50912,
    "Glucose": 50931,
    "Sodium": 50983,
    "Magnesium": 50960,
    "Potassium": 50971,
    "Phosphate": 50970,
    "Urea Nitrogen": 51006,
    "Hematocrit": 51221,
    "Hemoglobin": 51222,
    "MCH": 51248,
    "MCHC": 51249,
    "MCV": 51250,
    "Red Blood Cells": 51279,
    "White Blood Cells": 51301,
    "RDW": 51277,
    "Platelets": 51265
}

missing_percentages = calculate_missing_itemids_per_variable(labevents_file, chartevents_file, labevents_itemids, chartevents_itemids)

for item_name, percentage in missing_percentages.items():
  print(f"{item_name}: {percentage:.2f}% missing")

In [ ]:
# merge labevents and chartevents, only complete admissions

def merge_and_filter_files(labevents_file, chartevents_file, labevents_itemids, chartevents_itemids):

  labevents_df = pd.read_csv(labevents_file)
  chartevents_df = pd.read_csv(chartevents_file)

  labevents_filtered = labevents_df[labevents_df['itemid'].isin(labevents_itemids)]
  chartevents_filtered = chartevents_df[chartevents_df['itemid'].isin(chartevents_itemids)]

  labevents_grouped = labevents_filtered.groupby('hadm_id')['itemid'].nunique()
  chartevents_grouped = chartevents_filtered.groupby('hadm_id')['itemid'].nunique()

  labevents_complete_admissions = labevents_grouped[labevents_grouped == len(labevents_itemids)].index
  chartevents_complete_admissions = chartevents_grouped[chartevents_grouped == len(chartevents_itemids)].index

  complete_admissions = set(labevents_complete_admissions).intersection(set(chartevents_complete_admissions))

  labevents_complete = labevents_filtered[labevents_filtered['hadm_id'].isin(complete_admissions)]
  chartevents_complete = chartevents_filtered[chartevents_filtered['hadm_id'].isin(complete_admissions)]

  merged_df = pd.concat([labevents_complete, chartevents_complete], ignore_index=True)

  return merged_df

labevents_file = 'labevents_list.csv' 
chartevents_file = 'chartevents_list.csv' 
labevents_itemids = [50868, 50882, 50902, 50912, 50931, 50983, 50960, 50971, 50970, 
                      51006, 51221, 51222, 51248, 51249, 51250, 51279, 51301, 51277, 51265]
chartevents_itemids = [220045, 220277, 220210, 223761, 220739, 223900, 223901]

merged_df = merge_and_filter_files(labevents_file, chartevents_file, 
                                   labevents_itemids, chartevents_itemids)

merged_df.to_csv('measurements_complete.csv', index=False) 

In [ ]:
measurements_complete = pd.read_csv('measurements_complete.csv')

print('Number of measurements:', len(measurements_complete))
print('Unique admission ids:', measurements_complete['hadm_id'].nunique())

In [ ]:
print(measurements_complete.head())

In [ ]:
# add gender

patients = pd.read_csv('data/mimic-iv-3.1/mimic-iv-3.1/hosp/patients.csv')
measurements_complete = pd.read_csv('measurements_complete.csv')  

merged = pd.merge(measurements_complete, patients[['subject_id','gender']], on='subject_id', how='left')

merged.to_csv('data_with_gender.csv', index=False)

In [ ]:
# add age, first careunit, LOS

icustays = pd.read_csv('icustays_filtered.csv')
data_with_gender = pd.read_csv('data_with_gender.csv')  

merged = pd.merge(
    data_with_gender,
    icustays[['hadm_id', 'anchor_age', 'first_careunit', 'los']],
    on='hadm_id',
    how='left'
)

merged.to_csv('data_with_icustays_data.csv', index=False)

In [ ]:
# add admission type, admission location

admissions = pd.read_csv('data/mimic-iv-3.1/mimic-iv-3.1/hosp/admissions.csv')  

data_with_admissions = pd.read_csv('data_with_icustays_data.csv') 

merged = pd.merge(
    data_with_admissions,
    admissions[['hadm_id', 'admission_type', 'admission_location']],
    on='hadm_id',
    how='left'
)

merged.to_csv('data_with_admissions_data.csv', index=False)

In [ ]:
# add diagnoses

diagnoses = pd.read_csv('data/mimic-iv-3.1/mimic-iv-3.1/hosp/diagnoses_icd.csv')

data_with_admissions = pd.read_csv('data_with_admissions_data.csv') 

diagnoses_aggregated = (
    diagnoses.groupby('hadm_id')
    .apply(lambda x: [(code, version) for code, version in zip(x['icd_code'], x['icd_version'])]) 
    .reset_index()
    .rename(columns={0: 'diagnoses_with_version'})
)

merged = pd.merge(data_with_admissions, diagnoses_aggregated, on='hadm_id', how='left')

chunk_size = 100000  
with open('data_with_diagnoses.csv', 'w') as f:
    merged.iloc[:0].to_csv(f, index=False)
    for start in range(0, len(merged), chunk_size):
        merged.iloc[start:start + chunk_size].to_csv(f, index=False, header=False)

In [ ]:
# add los categories

data_with_diagnoses = pd.read_csv('data_with_diagnoses.csv')  

def categorize_los(los):
    if los <= 3:
        return 0
    elif 3 < los <= 7:
        return 1
    elif 7 < los <= 14:
        return 2
    else:
        return 3

data_with_diagnoses['los_class'] = data_with_diagnoses['los'].apply(categorize_los)

data_with_diagnoses.to_csv('data_with_los.csv', index=False)

In [ ]:
# filter out outliers

df = pd.read_csv('data_with_los.csv')

def filter_outliers(df):
    df['value'] = df['value'].apply(ast.literal_eval)

    expanded_df = df.explode('value')
    expanded_df['value'] = pd.to_numeric(expanded_df['value'], errors='coerce')

    iqr_bounds = expanded_df.groupby('itemid')['value'].agg(
        q1=lambda x: x.quantile(0.25),
        q3=lambda x: x.quantile(0.75)
    )
    iqr_bounds['iqr'] = iqr_bounds['q3'] - iqr_bounds['q1']
    iqr_bounds['lower_bound'] = iqr_bounds['q1'] - 1.5 * iqr_bounds['iqr']
    iqr_bounds['upper_bound'] = iqr_bounds['q3'] + 1.5 * iqr_bounds['iqr']

    expanded_df = expanded_df.merge(iqr_bounds[['lower_bound', 'upper_bound']], on='itemid', how='left')

    expanded_df = expanded_df[
        (expanded_df['value'] >= expanded_df['lower_bound']) & 
        (expanded_df['value'] <= expanded_df['upper_bound'])
    ]

    filtered_values = expanded_df.groupby(['itemid', 'hadm_id'], as_index=False).agg({'value': list})

    filtered_df = pd.merge(
        df.drop(columns=['value']),
        filtered_values,
        on=['itemid', 'hadm_id'],
        how='inner'
    )

    return filtered_df

measurements = pd.read_csv('data_with_los.csv')
print('loaded')

filtered_measurements = filter_outliers(measurements)
print('filtered')

filtered_measurements.to_csv('data_without_outliers.csv', index=False)


In [ ]:
# filter for complete admissions once more

required_itemids = [
    50868, 50882, 50902, 50912, 50931, 50983, 50960, 50971, 50970,
    51006, 51221, 51222, 51248, 51249, 51250, 51279, 51301, 51277, 51265,
    220045, 220277, 220210, 223761, 220739, 223900, 223901
]

data_without_outliers = pd.read_csv('data_without_outliers.csv')

filtered_measurements = data_without_outliers[data_without_outliers['itemid'].isin(required_itemids)]

hadm_itemid_count = (
    filtered_measurements.groupby('hadm_id')['itemid']
    .nunique()
    .reset_index()
    .rename(columns={'itemid': 'unique_itemid_count'})
)

valid_hadm_ids = hadm_itemid_count[hadm_itemid_count['unique_itemid_count'] == len(required_itemids)]['hadm_id']

final_filtered_measurements = data_without_outliers[data_without_outliers['hadm_id'].isin(valid_hadm_ids)]

final_filtered_measurements.to_csv('data_complete.csv', index=False)

In [ ]:
# calculate statistics

measurements = pd.read_csv('data_complete.csv')

def safe_eval(value):
    try:
        return literal_eval(value)  
    except (ValueError, SyntaxError):
        return [] 

measurements['value'] = measurements['value'].apply(safe_eval)

measurements['value'] = measurements['value'].apply(lambda x: [v for v in x if not pd.isna(v)])

measurements['min_value'] = measurements['value'].apply(lambda x: min(x) if x else np.nan)
measurements['max_value'] = measurements['value'].apply(lambda x: max(x) if x else np.nan)
measurements['mean_value'] = measurements['value'].apply(lambda x: np.mean(x) if x else np.nan)
measurements['sd_value'] = measurements['value'].apply(lambda x: np.std(x, ddof=1) if len(x) > 1 else np.nan)
measurements['skewness'] = measurements['value'].apply(lambda x: skew(x, bias=False) if len(x) > 1 else np.nan)
measurements['num_observations'] = measurements['value'].apply(lambda x: len(x))

measurements.to_csv('../data/variables_final.csv', index=False)